In [1]:
import pandas as pd
scans = pd.read_excel('Data Challenge Master File.xlsx', sheet_name = 'UCP Scans')
scans = scans.dropna(subset=['scan_datetime'])
scans.head()

,event_datetime,event_date,section_name,row_name,seat_number,scan_datetime
0,2022-10-23 19:00:00,2022-10-23,301,4,15,2022-10-23 19:38:42
1,2022-10-23 19:00:00,2022-10-23,318,10,23,2022-10-23 19:41:03
2,2022-10-23 19:00:00,2022-10-23,306,2,5,2022-10-23 19:44:19
3,2022-10-23 19:00:00,2022-10-23,316,2,2,2022-10-23 20:04:39
4,2022-10-23 19:00:00,2022-10-23,324,2,2,2022-10-23 20:14:35


In [2]:
import pandasql as ps
from pandasql import sqldf 
q = """
SELECT event_datetime, count(*) AS attendance
FROM scans
GROUP BY event_datetime
"""

df_scans = sqldf(q)
df_scans.head()

,event_datetime,attendance
0,2022-10-23 19:00:00.000000,244
1,2022-10-30 12:00:00.000000,194
2,2022-10-31 19:30:00.000000,91
3,2022-11-06 19:00:00.000000,136
4,2022-11-07 19:30:00.000000,179


In [3]:
import pandas as pd

# Convert event_datetime to datetime format
df_scans['event_datetime'] = pd.to_datetime(df_scans['event_datetime'])

# Extract the hour of the event and classify as afternoon or evening game
df_scans['game_time'] = df_scans['event_datetime'].apply(lambda x: 'Afternoon' if x.hour < 18 else 'Evening')

# Perform one-hot encoding
one_hot = pd.get_dummies(df_scans['game_time'])

# Add the one-hot encoded features to the original dataframe
df_scans = pd.concat([df_scans, one_hot], axis=1)

# Drop the original 'game_time' column
df_scans = df_scans.drop('game_time', axis=1)

# Print the resulting dataframe
df_scans.head()

,event_datetime,attendance,Afternoon,Evening
0,2022-10-23 19:00:00,244,0,1
1,2022-10-30 12:00:00,194,1,0
2,2022-10-31 19:30:00,91,0,1
3,2022-11-06 19:00:00,136,0,1
4,2022-11-07 19:30:00,179,0,1


In [4]:
df_scans = df_scans.drop('Afternoon', axis=1)
df_scans.head()

,event_datetime,attendance,Evening
0,2022-10-23 19:00:00,244,1
1,2022-10-30 12:00:00,194,0
2,2022-10-31 19:30:00,91,1
3,2022-11-06 19:00:00,136,1
4,2022-11-07 19:30:00,179,1


In [69]:

import urllib.request
import sys

try: 
  ResultBytes = urllib.request.urlopen("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/90015/2022-10-23/2023-04-08?unitGroup=metric&include=days&key=5KLJPVYWP7PJBDTDX8QW47DZU&contentType=xlsx")
  
  # Choose CSV or JSON to see sample parsing   
        
except urllib.error.HTTPError  as e:
  ErrorInfo= e.read().decode() 
  print('Error code: ', e.code, ErrorInfo)
  sys.exit()
except  urllib.error.URLError as e:
  ErrorInfo= e.read().decode() 
  print('Error code: ', e.code,ErrorInfo)
  sys.exit()

Error code:  429 You have exceeded the maximum number of daily result records for your account. Please add a credit card to continue retrieving results.


AssertionError: 

In [68]:
import pandas as pd
import json

# Get the data from the API
ResultBytes = urllib.request.urlopen("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/90015/2022-10-23/2023-04-09?unitGroup=metric&include=days&key=5KLJPVYWP7PJBDTDX8QW47DZU&contentType=json")

# Parse the data as JSON
jsonData = json.load(ResultBytes)

# Extract the windspeed, precip, and snow data for all the dates
data = []
for day in jsonData['days']:
    date = day['datetime']
    windspeed = day['windspeed']
    precip = day['precip']
    snow = day['snow']
    data.append([date, windspeed, precip, snow])

# Create a pandas dataframe from the extracted data
df_weather = pd.DataFrame(data, columns=['Date', 'WindSpeed', 'Precip', 'Snow'])

# Print the dataframe
print(df_weather)

HTTPError: HTTP Error 429: 

In [28]:
# ws = df['WindSpeed'].mean()
# ws

12.491666666666658

In [38]:
# import pandas as pd
# import numpy as np

# # Convert the WindSpeed, Precip, and Snow columns to numeric
# df['WindSpeed'] = pd.to_numeric(df['WindSpeed'], errors='coerce')
# df['Precip'] = pd.to_numeric(df['Precip'], errors='coerce')
# df['Snow'] = pd.to_numeric(df['Snow'], errors='coerce')

# # Fill any missing values with 0
# df.fillna(0, inplace=True)

# # Initialize empty lists for the encoded columns
# wind_13 = []
# precip_not_0 = []
# snow_not_0 = []

# # Iterate over the rows of the dataset and encode the values
# for i, row in df.iterrows():
#     # Encode WindSpeed
#     if row['WindSpeed'] > 13:
#         wind_13.append(1)
#     else:
#         wind_13.append(0)
    
#     # Encode Precip
#     if row['Precip'] != 0:
#         precip_not_0.append(1)
#     else:
#         precip_not_0.append(0)
    
#     # Encode Snow
#     if row['Snow'] != 0:
#         snow_not_0.append(1)
#     else:
#         snow_not_0.append(0)

# # Combine the encoded columns into a new DataFrame
# df_encoded = pd.DataFrame({'Date': df['Date'],
#                            'WindSpeed>13': wind_13,
#                            'Precip': precip_not_0,
#                            'Snow': snow_not_0})

# # Print the encoded DataFrame
# print(df_encoded)

           Date  WindSpeed>13  Precip  Snow
0    2022-10-23             0       0     0
1    2022-10-24             1       0     0
2    2022-10-25             0       0     0
3    2022-10-26             0       0     0
4    2022-10-27             0       0     0
..          ...           ...     ...   ...
163  2023-04-04             1       0     0
164  2023-04-05             0       0     0
165  2023-04-06             1       0     0
166  2023-04-07             1       0     0
167  2023-04-08             1       0     0

[168 rows x 4 columns]


In [63]:
df_scans['event_datetime'] = pd.to_datetime(df_scans['event_datetime'], format='%Y-%m-%d %H:%M:%S')
df_scans.head()

,event_datetime,attendance,Evening
0,2022-10-23,244,1
1,2022-10-30,194,0
2,2022-10-31,91,1
3,2022-11-06,136,1
4,2022-11-07,179,1


In [70]:
print(df_encoded)

          Date  WindSpeed>13  Precip  Snow
0   2022-10-23             0       0     0
1   2022-10-24             1       0     0
2   2022-10-25             0       0     0
3   2022-10-26             0       0     0
4   2022-10-27             0       0     0
..         ...           ...     ...   ...
163 2023-04-04             1       0     0
164 2023-04-05             0       0     0
165 2023-04-06             1       0     0
166 2023-04-07             1       0     0
167 2023-04-08             1       0     0

[168 rows x 4 columns]


In [71]:
df_encoded.to_excel('weather.xlsx')

In [74]:
df_weather.to_excel('Raw Weather.xlsx')

In [76]:
df_w = pd.read_excel('Raw Weather.xlsx')
df_w.head()

,Unnamed: 0,Date,WindSpeed,Precip,Snow
0,0,2022-10-23,11.0,0.0,0
1,1,2022-10-24,19.4,0.0,0
2,2,2022-10-25,6.8,0.0,0
3,3,2022-10-26,10.4,0.0,0
4,4,2022-10-27,12.3,0.0,0


In [89]:
q = """
SELECT *
FROM df_scans s
JOIN df_weather e ON s.event_datetime = e.Date
"""

df_eve_weather = sqldf(q)
df_eve_weather = df_eve_weather.drop('Date', axis =1)
df_eve_weather.head()

,event_datetime,attendance,Evening,WindSpeed,Precip,Snow
0,2022-10-23,244,1,11.0,0.000,0.0
1,2022-10-30,194,0,10.1,0.000,0.0
2,2022-10-31,91,1,8.2,0.000,0.0
3,2022-11-06,136,1,9.9,0.000,0.0
4,2022-11-07,179,1,10.0,1.879,0.0


In [109]:
import pandas as pd

# Convert the 'event_datetime' column to datetime format
df_eve_weather['event_datetime'] = pd.to_datetime(df_eve_weather['event_datetime'])

# Create a new column 'leisure_day' that indicates whether the date is on a Friday or Saturday or not
df_eve_weather['Friday_or_Saturday'] = df_eve_weather['event_datetime'].apply(lambda x: x.weekday() in [4, 5]).astype(int)
df_eve_weather.head()

,event_datetime,attendance,Evening,WindSpeed,Precip,Snow,Friday_or_Saturday
0,2022-10-23,244,1,11.0,0.000,0.0,0
1,2022-10-30,194,0,10.1,0.000,0.0,0
2,2022-10-31,91,1,8.2,0.000,0.0,0
3,2022-11-06,136,1,9.9,0.000,0.0,0
4,2022-11-07,179,1,10.0,1.879,0.0,0


In [110]:
sports_refer = pd.read_excel('Sports Reference.xlsx')

# assuming the date column in sports_refer is named 'Date'
sports_refer['Date'] = pd.to_datetime(sports_refer['Date'])
sports_refer['Date'] = sports_refer['Date'].dt.strftime('%Y-%m-%d')

# substitute home_stands with 'H' as 1, others as 0
sports_refer['Home_Stand'] = sports_refer['Home_Stand'].apply(lambda x: 1 if x == 'H' else 0)

# keep only the date and home_stands columns
sports_refer = sports_refer[['Date', 'Home_Stand']]
sports_refer.head()


,Date,Home_Stand
0,2022-10-20,0
1,2022-10-22,0
2,2022-10-23,0
3,2022-10-25,0
4,2022-10-27,0


In [118]:
df_eve_weather['event_datetime'] = pd.to_datetime(df_eve_weather['event_datetime'])
df_eve_weather['event_datetime'] = df_eve_weather['event_datetime'].dt.strftime('%Y-%m-%d')


In [119]:
q = """
SELECT *
FROM df_eve_weather d
JOIN sports_refer r ON d.event_datetime = r.Date
"""

df_ewls= sqldf(q)
df_ewls = df_ewls.drop('Date', axis =1)
df_ewls.head()

,event_datetime,attendance,Evening,WindSpeed,Precip,Snow,Friday_or_Saturday,Home_Stand
0,2022-10-23,244,1,11.0,0.000,0.0,0,0
1,2022-10-30,194,0,10.1,0.000,0.0,0,0
2,2022-10-31,91,1,8.2,0.000,0.0,0,0
3,2022-11-06,136,1,9.9,0.000,0.0,0,1
4,2022-11-07,179,1,10.0,1.879,0.0,0,1


In [141]:
sports_refer2 = pd.read_excel('Sports Reference.xlsx')

In [124]:
famous_leagues = [
    'Los Angeles Lakers',
    'Phoenix Suns',
    'Oklahoma City Thunder',
    'Houston Rockets',
    'San Antonio Spurs',
    'Cleveland Cavaliers',
    'Brooklyn Nets',
    'Dallas Mavericks',
    'Golden State Warriors',
    'Portland Trail Blazers',
    'Miami Heat',
    'Boston Celtics',
    'Philadelphia 76ers',
    'Toronto Raptors',
    'Milwaukee Bucks',
    'New York Knicks'
]


# Define a function to check if opponent is in famous leagues list
def check_fame_opponent(opponent):
    if any(league in opponent for league in famous_leagues):
        return 1
    else:
        return 0

# Apply the function to "Opponent" column and create a new column "fame_opponent"
sports_refer2['fame_opponent'] = sports_refer2['Opponent'].apply(check_fame_opponent)
# sports_refer2.head()

In [127]:
sports_refer2 = sports_refer2[['Date', 'fame_opponent']]

In [128]:
sports_refer2.head()

,Date,fame_opponent
0,"Thu, Oct 20, 2022",1
1,"Sat, Oct 22, 2022",0
2,"Sun, Oct 23, 2022",1
3,"Tue, Oct 25, 2022",1
4,"Thu, Oct 27, 2022",1


In [131]:
df_ewls['event_datetime'] = pd.to_datetime(df_ewls['event_datetime'])
df_ewls['event_datetime'] = df_ewls['event_datetime'].dt.strftime('%Y-%m-%d')

In [134]:
sports_refer2['Date'] = pd.to_datetime(sports_refer['Date'])
sports_refer2['Date'] = sports_refer2['Date'].dt.strftime('%Y-%m-%d')

In [138]:
q = """
SELECT *
FROM df_ewls d
JOIN sports_refer2 r ON d.event_datetime = r.Date
"""

df_ewlso= sqldf(q)
df_ewlso = df_ewlso.drop('Date', axis=1)
df_ewlso.head()

,event_datetime,attendance,Evening,WindSpeed,Precip,Snow,Friday_or_Saturday,Home_Stand,fame_opponent
0,2022-10-23,244,1,11.0,0.000,0.0,0,0,1
1,2022-10-30,194,0,10.1,0.000,0.0,0,0,0
2,2022-10-31,91,1,8.2,0.000,0.0,0,0,1
3,2022-11-06,136,1,9.9,0.000,0.0,0,1,0
4,2022-11-07,179,1,10.0,1.879,0.0,0,1,1


In [144]:
sports_refer2 = sports_refer2[['Date', 'Win_Stand']]
sports_refer2.head()

,Date,Win_Stand
0,"Thu, Oct 20, 2022",NaN
1,"Sat, Oct 22, 2022",NaN
2,"Sun, Oct 23, 2022",1.0
3,"Tue, Oct 25, 2022",NaN
4,"Thu, Oct 27, 2022",NaN


In [147]:
sports_refer2['Date'] = pd.to_datetime(sports_refer['Date'])
sports_refer2['Date'] = sports_refer2['Date'].dt.strftime('%Y-%m-%d')

In [148]:
df_ewlso['event_datetime'] = pd.to_datetime(df_ewlso['event_datetime'])
df_ewlso['event_datetime'] = df_ewlso['event_datetime'].dt.strftime('%Y-%m-%d')

In [157]:
q = """
SELECT *
FROM df_ewlso d
JOIN sports_refer2 r ON d.event_datetime = r.Date
"""

df_ewlsow= sqldf(q)
df_ewlsow.head()

,event_datetime,attendance,Evening,WindSpeed,Precip,Snow,Friday_or_Saturday,Home_Stand,fame_opponent,Date,Win_Stand
0,2022-10-23,244,1,11.0,0.000,0.0,0,0,1,2022-10-23,1.0
1,2022-10-30,194,0,10.1,0.000,0.0,0,0,0,2022-10-30,NaN
2,2022-10-31,91,1,8.2,0.000,0.0,0,0,1,2022-10-31,NaN
3,2022-11-06,136,1,9.9,0.000,0.0,0,1,0,2022-11-06,NaN
4,2022-11-07,179,1,10.0,1.879,0.0,0,1,1,2022-11-07,NaN


In [158]:
df_ewlsow.fillna(0, inplace=True)
df_ewlsow.drop('Date', axis=1, inplace=True)
df_ewlsow.head()

,event_datetime,attendance,Evening,WindSpeed,Precip,Snow,Friday_or_Saturday,Home_Stand,fame_opponent,Win_Stand
0,2022-10-23,244,1,11.0,0.000,0.0,0,0,1,1.0
1,2022-10-30,194,0,10.1,0.000,0.0,0,0,0,0.0
2,2022-10-31,91,1,8.2,0.000,0.0,0,0,1,0.0
3,2022-11-06,136,1,9.9,0.000,0.0,0,1,0,0.0
4,2022-11-07,179,1,10.0,1.879,0.0,0,1,1,0.0


In [156]:
df_ewlsow.to_excel('Data Prediction Model.xlsx')